In [2]:
import pandas as pd

In [3]:
dataset = pd.read_csv('dataset_min.csv',index_col=0)
submission = pd.read_csv('submission_min.csv',index_col=0)

In [9]:
submission

,사용년,사용월,사용지역_도1,업종명_대분류1,업종명1,이용금액
0,1,4,4,1,31,0
1,1,4,4,3,23,0
2,1,4,4,1,13,0
3,1,4,4,1,32,0
4,1,4,4,18,40,0
...,...,...,...,...,...,...
1389,1,7,8,0,7,0
1390,1,7,8,0,0,0
1391,1,7,8,14,27,0
1392,1,7,8,1,15,0


In [11]:
submission.iloc[0,4] = 333

In [12]:
submission

,사용년,사용월,사용지역_도1,업종명_대분류1,업종명1,이용금액
0,1,4,4,1,333,0
1,1,4,4,3,23,0
2,1,4,4,1,13,0
3,1,4,4,1,32,0
4,1,4,4,18,40,0
...,...,...,...,...,...,...
1389,1,7,8,0,7,0
1390,1,7,8,0,0,0
1391,1,7,8,14,27,0
1392,1,7,8,1,15,0


In [8]:
dataset[dataset['이용금액'].isnull()]

,사용년,사용월,사용지역_도1,업종명_대분류1,업종명1,이용금액


In [75]:
dataset_cor = dataset.corr()

In [76]:
dataset_cor

,사용년,사용월,사용지역_도1,업종명_대분류1,업종명1,이용금액
사용년,1.000000,-0.499500,-0.002615,-0.005513,-0.008526,-0.013327
사용월,-0.499500,1.000000,0.001876,0.000256,0.002351,0.011533
사용지역_도1,-0.002615,0.001876,1.000000,-0.028137,-0.044994,-0.256963
업종명_대분류1,-0.005513,0.000256,-0.028137,1.000000,0.589526,-0.187674
업종명1,-0.008526,0.002351,-0.044994,0.589526,1.000000,-0.370045
이용금액,-0.013327,0.011533,-0.256963,-0.187674,-0.370045,1.000000


In [158]:
x = dataset.iloc[:,:-1]
y = dataset.iloc[:,-1]

In [78]:
x

,사용년,사용월,사용지역_도1,업종명_대분류1,업종명1
0,0,1,0,0,0
1,0,1,0,0,6
2,0,1,0,0,7
3,0,1,0,0,9
4,0,1,0,0,11
...,...,...,...,...,...
9433,1,3,16,11,21
9434,1,3,16,12,26
9435,1,3,16,12,33
9436,1,3,16,13,24


In [79]:
test = submission.iloc[:,:-1]

# 머신러닝

In [80]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from xgboost import XGBRegressor

In [166]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV

In [159]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, random_state=66, test_size=0.2)

In [171]:
pipe = Pipeline([("scaler",RobustScaler()),('rf',RandomForestRegressor())]) 

In [172]:
parameters = [
    { 'rf__max_depth':[3,5,25,100, 200, 300],
     'rf__n_estimators':[10, 50, 150, 250, 300],
    'rf__max_leaf_nodes':[1000000,1005000,10000000]}
]

In [173]:
model = RandomizedSearchCV(pipe, parameters, cv=5)
model.fit(x_train, y_train)
pred = model.predict(x_test)
RMSLE = np.sqrt(mean_squared_log_error(y_test, pred))
MAE = mean_absolute_error(y_test, pred)
print('rmsle는',RMSLE)
print('mae는',MAE)

rmsle는 0.4644819596117981
mae는 488800078.0807945


In [175]:
test_pred = model.predict(test)
real_submission = pd.read_csv('submission1.csv')
real_submission['AMT'] = test_pred
real_submission.to_csv('submission_0630_3.csv', index=None, encoding='utf-8-sig')

In [174]:
print("============================")
print("최적의 매개 변수는(estimator):", model.best_estimator_) # 전체 params 다 나옴
print("============================")
print("최적의 매개 변수는(params):", model.best_params_)
print("============================")

최적의 매개 변수는(estimator): Pipeline(memory=None,
         steps=[('scaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('rf',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=100,
                                       max_features='auto',
                                       max_leaf_nodes=1005000, max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=300, n_jobs=None,
                                       oob_score=False, random_state=None,
                                       verbose=0, warm_st

In [116]:
model = RandomForestRegressor()
model.fit(x_train, y_train)
pred = model.predict(x_test)
RMSLE = np.sqrt(mean_squared_log_error(y_test, pred))
RMSLE

0.4715094243268884

In [130]:
model = RandomForestRegressor(n_estimators=150)
model.fit(x_train, y_train)
pred = model.predict(x_test)
RMSLE = np.sqrt(mean_squared_log_error(y_test, pred))
MAE = mean_absolute_error(y_test, pred)
print('rmsle는',RMSLE)
print('mae는',MAE)

rmsle는 0.4647577750664943
mae는 492503686.91604173


In [120]:
model = XGBRegressor(n_estimators=150)
model.fit(x_train, y_train)
pred = model.predict(x_test)
RMSLE = np.sqrt(mean_squared_log_error(y_test, pred))
MAE = mean_absolute_error(y_test, pred)
print('rmsle는',RMSLE)
print('mae는',MAE)

ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [91]:
test_pred = model.predict(test)
real_submission = pd.read_csv('submission1.csv')
real_submission['AMT'] = test_pred
real_submission.to_csv('submission_0630_2.csv', index=None, encoding='utf-8-sig')

In [102]:
submission_real

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,9.751899e+07
1,1,202004,강원,골프장 운영업,3.760515e+09
2,2,202004,강원,과실 및 채소 소매업,9.334674e+08
3,3,202004,강원,관광 민예품 및 선물용품 소매업,1.834808e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.619220e+06
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.462200e+09
1390,1390,202007,충북,한식 음식점업,2.084806e+10
1391,1391,202007,충북,호텔업,1.708192e+07
1392,1392,202007,충북,화장품 및 방향제 소매업,4.778178e+08


In [105]:
submission_real.to_csv('submission_0630_1.csv', index=None, encoding='utf-8-sig')

In [106]:
submission_real

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,9.751899e+07
1,1,202004,강원,골프장 운영업,3.760515e+09
2,2,202004,강원,과실 및 채소 소매업,9.334674e+08
3,3,202004,강원,관광 민예품 및 선물용품 소매업,1.834808e+07
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.619220e+06
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.462200e+09
1390,1390,202007,충북,한식 음식점업,2.084806e+10
1391,1391,202007,충북,호텔업,1.708192e+07
1392,1392,202007,충북,화장품 및 방향제 소매업,4.778178e+08


In [62]:
submission_real = submission_real.reset_index()

In [107]:
submission_real.to_csv('submission_0630_1.csv', index=None, encoding='utf-8-sig')

In [ ]:
# xgb 완전 쓰레기...

In [40]:
x_train.shape

(7550, 5)

In [110]:
x_test.shape

(1888, 5)

In [118]:
from keras.models import Sequential, Model 
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, LSTM

model = Sequential()
model.add(Dense(1600, input_shape=(5,), activation='elu'))
model.add(Dense(3600, activation='elu'))
model.add(Dropout(0.2))

model.add(Dense(1280, activation='elu'))
model.add(Dense(640, activation='elu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='elu'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 1600)              9600      
_________________________________________________________________
dense_17 (Dense)             (None, 3600)              5763600   
_________________________________________________________________
dropout_7 (Dropout)          (None, 3600)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 1280)              4609280   
_________________________________________________________________
dense_19 (Dense)             (None, 640)               819840    
_________________________________________________________________
dropout_8 (Dropout)          (None, 640)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                

In [119]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit(x_train, y_train, epochs=500, batch_size=100, validation_split=0.2)

Train on 6040 samples, validate on 1510 samples
Epoch 1/500
6040/6040 [==============================] - 1s 116us/step - loss: 402215335456531480576.0000 - mse: 402215321636180066304.0000 - val_loss: 369562660599200153600.0000 - val_mse: 369562641040999972864.0000
Epoch 2/500
6040/6040 [==============================] - 1s 86us/step - loss: 396847060079305621504.0000 - mse: 396846995695982346240.0000 - val_loss: 358979894240020332544.0000 - val_mse: 358979920788493172736.0000
Epoch 3/500
6040/6040 [==============================] - 1s 88us/step - loss: 386215249669018288128.0000 - mse: 386215298797527891968.0000 - val_loss: 350154815819465555968.0000 - val_mse: 350154800659311886336.0000
Epoch 4/500
6040/6040 [==============================] - 1s 87us/step - loss: 370406392953511149568.0000 - mse: 370406432652434341888.0000 - val_loss: 326204603076538662912.0000 - val_mse: 326204587472211410944.0000
Epoch 5/500
6040/6040 [==============================] - 1s 87us/step - loss: 331049644

Epoch 39/500
6040/6040 [==============================] - 1s 86us/step - loss: 114655107313077305344.0000 - mse: 114655111015737327616.0000 - val_loss: 95393270909325557760.0000 - val_mse: 95393268186031849472.0000
Epoch 40/500
6040/6040 [==============================] - 1s 92us/step - loss: 113993319379588530176.0000 - mse: 113993328161118486528.0000 - val_loss: 96313052419833872384.0000 - val_mse: 96313049245085073408.0000
Epoch 41/500
6040/6040 [==============================] - 1s 86us/step - loss: 114248800248510349312.0000 - mse: 114248810682948517888.0000 - val_loss: 94561087941657247744.0000 - val_mse: 94561096213479817216.0000
Epoch 42/500
6040/6040 [==============================] - 1s 87us/step - loss: 113198707030065758208.0000 - mse: 113198689117492215808.0000 - val_loss: 91523588812906807296.0000 - val_mse: 91523594186678861824.0000
Epoch 43/500
6040/6040 [==============================] - 1s 88us/step - loss: 112371443885428195328.0000 - mse: 112371451753125642240.0000 

6040/6040 [==============================] - 1s 89us/step - loss: 75043813051152056320.0000 - mse: 75043806046316396544.0000 - val_loss: 69893097573696028672.0000 - val_mse: 69893091049441591296.0000
Epoch 116/500
6040/6040 [==============================] - 1s 86us/step - loss: 74432216429057507328.0000 - mse: 74432213698482274304.0000 - val_loss: 71830582381230931968.0000 - val_mse: 71830575673117769728.0000
Epoch 117/500
6040/6040 [==============================] - 1s 86us/step - loss: 74380893614616068096.0000 - mse: 74380870903511646208.0000 - val_loss: 70588034868929732608.0000 - val_mse: 70588026378661134336.0000
Epoch 118/500
6040/6040 [==============================] - 1s 86us/step - loss: 71583644078702313472.0000 - mse: 71583647351751835648.0000 - val_loss: 68728612334183800832.0000 - val_mse: 68728611478603563008.0000
Epoch 119/500
6040/6040 [==============================] - 1s 87us/step - loss: 69279708267066941440.0000 - mse: 69279713094723960832.0000 - val_loss: 6776036

6040/6040 [==============================] - 1s 86us/step - loss: 38127589009333157888.0000 - mse: 38127588794527907840.0000 - val_loss: 39609309188341235712.0000 - val_mse: 39609308256304889856.0000
Epoch 192/500
6040/6040 [==============================] - 1s 85us/step - loss: 38807101993196118016.0000 - mse: 38807104572679520256.0000 - val_loss: 39066072903164764160.0000 - val_mse: 39066074745392857088.0000
Epoch 193/500
6040/6040 [==============================] - 1s 86us/step - loss: 39560878388023738368.0000 - mse: 39560881366171123712.0000 - val_loss: 39666932168002166784.0000 - val_mse: 39666931461693374464.0000
Epoch 194/500
6040/6040 [==============================] - 1s 87us/step - loss: 38752485888396173312.0000 - mse: 38752485233058119680.0000 - val_loss: 38986879784486993920.0000 - val_mse: 38986879121867407360.0000
Epoch 195/500
6040/6040 [==============================] - 1s 86us/step - loss: 39810702619134279680.0000 - mse: 39810708000187875328.0000 - val_loss: 3956211

6040/6040 [==============================] - 1s 87us/step - loss: 30061468204603965440.0000 - mse: 30061476935163183104.0000 - val_loss: 30275941236401152000.0000 - val_mse: 30275943275230658560.0000
Epoch 268/500
6040/6040 [==============================] - 1s 87us/step - loss: 30705245442243280896.0000 - mse: 30705247590295797760.0000 - val_loss: 30131728519459450880.0000 - val_mse: 30131726932085047296.0000
Epoch 269/500
6040/6040 [==============================] - 1s 87us/step - loss: 31128270460965257216.0000 - mse: 31128271494943080448.0000 - val_loss: 29850102480045285376.0000 - val_mse: 29850102421793013760.0000
Epoch 270/500
6040/6040 [==============================] - 1s 88us/step - loss: 30614876097334362112.0000 - mse: 30614878729608888320.0000 - val_loss: 29891023186334572544.0000 - val_mse: 29891024045555580928.0000
Epoch 271/500
6040/6040 [==============================] - 1s 88us/step - loss: 30424541625488445440.0000 - mse: 30424544470747840512.0000 - val_loss: 3061787

6040/6040 [==============================] - 1s 88us/step - loss: 28369523422990794752.0000 - mse: 28369522054062407680.0000 - val_loss: 27196968077595893760.0000 - val_mse: 27196969868853248000.0000
Epoch 344/500
6040/6040 [==============================] - 1s 86us/step - loss: 27314298346887323648.0000 - mse: 27314300953676480512.0000 - val_loss: 26432744234324541440.0000 - val_mse: 26432745515874516992.0000
Epoch 345/500
6040/6040 [==============================] - 1s 88us/step - loss: 26749381378086223872.0000 - mse: 26749382874441449472.0000 - val_loss: 26517433250951446528.0000 - val_mse: 26517434299492335616.0000
Epoch 346/500
6040/6040 [==============================] - 1s 88us/step - loss: 27891879122899681280.0000 - mse: 27891872212723957760.0000 - val_loss: 28339641556770304000.0000 - val_mse: 28339639527042711552.0000
Epoch 347/500
6040/6040 [==============================] - 1s 88us/step - loss: 26420412730837790720.0000 - mse: 26420411194434125824.0000 - val_loss: 2644880

KeyboardInterrupt: 

In [ ]:
x_train = x_train.reshape(7550,5,1)
x_test = x_test.reshape(1888,5,1)

In [46]:
from keras.models import Sequential, Model 
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, LSTM

model = Sequential()
model.add(LSTM(160, input_shape=(5,)))
model.add(Dense(160))
model.add(Dropout(0.2))

model.add(Dense(128))
model.add(Dense(64))
model.add(Dropout(0.2))

model.add(Dense(1, activation='relu'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 16)                96        
_________________________________________________________________
dense_12 (Dense)             (None, 16)                272       
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 64)                1088      
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                

In [48]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit(x_train, y_train, epochs=50, batch_size=2, validation_split=0.2)

Train on 6040 samples, validate on 1510 samples
Epoch 1/50
6040/6040 [==============================] - 9s 1ms/step - loss: 389524191217854513152.0000 - mse: 389523966780017475584.0000 - val_loss: 385864514416192389120.0000 - val_mse: 385864651345290592256.0000
Epoch 2/50
6040/6040 [==============================] - 9s 1ms/step - loss: 385136523063873503232.0000 - mse: 385136299658679681024.0000 - val_loss: 380919034811891974144.0000 - val_mse: 380918924898251833344.0000
Epoch 3/50
6040/6040 [==============================] - 9s 1ms/step - loss: 377712754277390483456.0000 - mse: 377712608254168662016.0000 - val_loss: 371906013596131655680.0000 - val_mse: 371906096143976628224.0000
Epoch 4/50
6040/6040 [==============================] - 9s 1ms/step - loss: 364756766988976455680.0000 - mse: 364756174338688811008.0000 - val_loss: 359675310217679077376.0000 - val_mse: 359675445455945203712.0000
Epoch 5/50
6040/6040 [==============================] - 9s 1ms/step - loss: 35323226148731525529

KeyboardInterrupt: 